Machine Learning with Python

Link to the video series 

https://www.youtube.com/watch?v=OGxgnH8y2NM&list=PLQVvvaa0QuDfKTOs3Keq_kaG2P55YRn5v

In [1]:
import pandas as pd
import numpy as np
import quandl, math
from sklearn import preprocessing, svm
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
df = quandl.get("WARSAWSE/INVESTORMS", authtoken="sYa-65X6FfYXxwSf26QF")

df.head(10)

,Open,High,Low,Close,% Change,Turnover (1000s)
Date,,,,,,
2016-10-03,4167.30,4181.60,4167.30,4181.60,0.13,122923.48
2016-10-04,4189.68,4189.68,4185.27,4185.27,0.09,202175.81
2016-10-05,4171.36,4171.36,4168.79,4168.79,-0.39,172844.80
2016-10-06,4167.50,4169.02,4167.50,4169.02,0.01,139656.10
2016-10-07,4169.35,4169.35,4146.10,4146.10,-0.55,159333.27
2016-10-10,4140.94,4150.22,4140.94,4150.22,0.10,106643.22
2016-10-11,4151.86,4151.86,4146.02,4146.02,-0.10,276185.64
2016-10-12,4122.87,4135.40,4122.87,4135.40,-0.26,188901.94
2016-10-13,4137.46,4137.46,4123.86,4123.86,-0.28,219263.28


In [3]:
print(df.shape)
print(np.product(df.shape))

(273, 6)
1638


In [4]:
df['HL_%CHNG'] = ((df['High'] - df['Low']) / df['Low']) * 100
df['Daily_%CHNG'] = ((df['Close'] - df['Open']) / df['Open']) * 100

df.head(10)

,Open,High,Low,Close,% Change,Turnover (1000s),HL_%CHNG,Daily_%CHNG
Date,,,,,,,,
2016-10-03,4167.30,4181.60,4167.30,4181.60,0.13,122923.48,0.343148,0.343148
2016-10-04,4189.68,4189.68,4185.27,4185.27,0.09,202175.81,0.105370,-0.105259
2016-10-05,4171.36,4171.36,4168.79,4168.79,-0.39,172844.80,0.061649,-0.061611
2016-10-06,4167.50,4169.02,4167.50,4169.02,0.01,139656.10,0.036473,0.036473
2016-10-07,4169.35,4169.35,4146.10,4146.10,-0.55,159333.27,0.560768,-0.557641
2016-10-10,4140.94,4150.22,4140.94,4150.22,0.10,106643.22,0.224104,0.224104
2016-10-11,4151.86,4151.86,4146.02,4146.02,-0.10,276185.64,0.140858,-0.140660
2016-10-12,4122.87,4135.40,4122.87,4135.40,-0.26,188901.94,0.303915,0.303915
2016-10-13,4137.46,4137.46,4123.86,4123.86,-0.28,219263.28,0.329788,-0.328704


In [5]:
df.isnull().sum()

Open                0
High                0
Low                 0
Close               0
% Change            0
Turnover (1000s)    0
HL_%CHNG            0
Daily_%CHNG         0
dtype: int64

In [6]:
forecast_out = int( math.ceil(0.036*len(df)) )
print(forecast_out)

# We cannot use Close price as our label, since on that particular day, 
# if we dont know Close price then we dont know % changes,
# our model will be biased via Data Leakage,
# Basically, we will now be training our model on 273 
# days of stock price data instances
# wherein our label value will be one that occured after 10 days (on 11th day)

10


In [7]:
df['label'] = df['Close'].shift(-forecast_out)

df.head(20)

,Open,High,Low,Close,% Change,Turnover (1000s),HL_%CHNG,Daily_%CHNG,label
Date,,,,,,,,,
2016-10-03,4167.30,4181.60,4167.30,4181.60,0.13,122923.48,0.343148,0.343148,4159.98
2016-10-04,4189.68,4189.68,4185.27,4185.27,0.09,202175.81,0.105370,-0.105259,4175.00
2016-10-05,4171.36,4171.36,4168.79,4168.79,-0.39,172844.80,0.061649,-0.061611,4205.95
2016-10-06,4167.50,4169.02,4167.50,4169.02,0.01,139656.10,0.036473,0.036473,4205.01
2016-10-07,4169.35,4169.35,4146.10,4146.10,-0.55,159333.27,0.560768,-0.557641,4201.09
2016-10-10,4140.94,4150.22,4140.94,4150.22,0.10,106643.22,0.224104,0.224104,4217.49
2016-10-11,4151.86,4151.86,4146.02,4146.02,-0.10,276185.64,0.140858,-0.140660,4190.29
2016-10-12,4122.87,4135.40,4122.87,4135.40,-0.26,188901.94,0.303915,0.303915,4192.92
2016-10-13,4137.46,4137.46,4123.86,4123.86,-0.28,219263.28,0.329788,-0.328704,4202.64


In [8]:
df.columns

Index(['Open', 'High', 'Low', 'Close', '% Change', 'Turnover (1000s)',
       'HL_%CHNG', 'Daily_%CHNG', 'label'],
      dtype='object')

In [9]:
#Filter out the imp columns 

df = df[['Close', '% Change', 'HL_%CHNG','Turnover (1000s)', 'label']]
df.head(10)

,Close,% Change,HL_%CHNG,Turnover (1000s),label
Date,,,,,
2016-10-03,4181.60,0.13,0.343148,122923.48,4159.98
2016-10-04,4185.27,0.09,0.105370,202175.81,4175.00
2016-10-05,4168.79,-0.39,0.061649,172844.80,4205.95
2016-10-06,4169.02,0.01,0.036473,139656.10,4205.01
2016-10-07,4146.10,-0.55,0.560768,159333.27,4201.09
2016-10-10,4150.22,0.10,0.224104,106643.22,4217.49
2016-10-11,4146.02,-0.10,0.140858,276185.64,4190.29
2016-10-12,4135.40,-0.26,0.303915,188901.94,4192.92
2016-10-13,4123.86,-0.28,0.329788,219263.28,4202.64


In [10]:
# df.drop(df.index[[262,263,264,265,266,267,268,269,270,271,272]], inplace=True)
df.tail(20)

,Close,% Change,HL_%CHNG,Turnover (1000s),label
Date,,,,,
2017-10-05,4757.99,-0.22,0.009878,194805.65,4649.64
2017-10-06,4755.31,-0.06,0.122810,217121.56,4647.10
2017-10-09,4745.97,-0.20,0.227140,142719.07,4613.26
2017-10-10,4740.97,-0.11,0.086058,206002.90,4556.59
2017-10-11,4779.66,0.82,0.445944,389146.74,4572.41
2017-10-12,4777.99,-0.03,0.209502,262822.20,4563.95
2017-10-13,4755.66,-0.47,0.345693,161286.71,4601.19
2017-10-16,4755.10,-0.01,0.382537,182020.51,4625.71
2017-10-17,4716.42,-0.81,0.579889,171865.20,4637.02


In [11]:
df.dropna(inplace=True)
y = np.array(df['label'])
X = np.array(df.drop(['label'],1))
X = preprocessing.scale(X)

In [12]:
X_train, X_test, y_train, y_test = train_test_split( X,y, test_size=0.8, random_state=0 )

clf = LinearRegression()
clf.fit(X_train, y_train)

score = clf.score(X_test, y_test)
print(score)

0.9095614908371158


In [13]:
clf2 = svm.SVR()
clf2.fit(X_train, y_train)
score2 = clf2.score(X_test, y_test)

print(score2)

-0.16769394785458602
